In [1]:
import json
json_path = '/home/pgajo/working/food/data/TASTEset/data/formatted data/TASTEset_sep_format.json'
with open(json_path) as json_file:
    data = json.load(json_file)

recipe_list = data['annotations']
recipe_list

[{'text_en': '5 ounces rum;4 ounces triple sec;3 ounces Tia Maria;20 ounces orange juice',
  'entities_en': [[0, 1, 'QUANTITY'],
   [2, 8, 'UNIT'],
   [9, 12, 'FOOD'],
   [13, 14, 'QUANTITY'],
   [15, 21, 'UNIT'],
   [22, 32, 'FOOD'],
   [33, 34, 'QUANTITY'],
   [35, 41, 'UNIT'],
   [42, 51, 'FOOD'],
   [52, 54, 'QUANTITY'],
   [55, 61, 'UNIT'],
   [62, 74, 'FOOD']]},
 {'text_en': '2 tubes cinnamon roll, refrigerated, with icing;4 tablespoons butter, melted;6 eggs;1/2 cup milk;2 teaspoons cinnamon;2 teaspoons vanilla;1 cup maple syrup',
  'entities_en': [[0, 1, 'QUANTITY'],
   [2, 7, 'UNIT'],
   [8, 21, 'FOOD'],
   [23, 35, 'PROCESS'],
   [37, 41, 'FOOD'],
   [42, 47, 'FOOD'],
   [48, 49, 'QUANTITY'],
   [50, 61, 'UNIT'],
   [62, 68, 'FOOD'],
   [70, 76, 'PROCESS'],
   [77, 78, 'QUANTITY'],
   [79, 83, 'FOOD'],
   [84, 87, 'QUANTITY'],
   [88, 91, 'UNIT'],
   [92, 96, 'FOOD'],
   [97, 98, 'QUANTITY'],
   [99, 108, 'UNIT'],
   [109, 117, 'FOOD'],
   [118, 119, 'QUANTITY'],
   [120, 129,

In [2]:
import deepl
from tqdm.auto import tqdm
auth_key = "c8fe40ff-eada-719c-616f-ee298486d413:fx"
translator = deepl.Translator(auth_key)

/home/pgajo/working/food/food-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tqdm.auto import tqdm
pbar = tqdm(recipe_list, total=len(recipe_list))
for recipe in pbar:
    pbar.set_description(f"DeepL usage: {translator.get_usage().character.count}")
    recipe['text_it'] = ''
    recipe['entities_it'] = []
    # print('-----------------------------------')
    # print("recipe['text_en']", recipe['text_en'])
    entity_translations = translator.translate_text([recipe['text_en'][entity[0]:entity[1]] for entity in recipe['entities_en']], target_lang="IT")
    for i, entity in enumerate(recipe['entities_en']):
        original_text = recipe['text_en'][entity[0]:entity[1]]
        # print("original_text", original_text)
        translated_text = entity_translations[i].text
        # print("translated_text", translated_text)
        recipe['text_it'] += translated_text
        # print("recipe['text_it']", recipe['text_it'])
        recipe['entities_it'].append([len(recipe['text_it']) - len(translated_text), len(recipe['text_it']), entity[2]])
        sep_len = 0
        # find the next alphanumeric character after the entity and put whatever is in between the entity and the next alphanumeric character in the separator
        while entity[1]+sep_len < len(recipe['text_en']) and not recipe['text_en'][entity[1]+sep_len].isalnum():
            sep_len += 1
        separator = recipe['text_en'][entity[1]:entity[1]+sep_len] # use the separator from the original text
        if separator == '':
            separator = ' '
        recipe['text_it'] += separator 
    recipe['text_it'] = recipe['text_it'].strip()
    # print("recipe['text_it']", recipe['text_it'])
print(recipe_list)

100%|██████████| 700/700 [04:19<00:00,  2.70it/s]

207867
[{'text_en': '5 ounces rum;4 ounces triple sec;3 ounces Tia Maria;20 ounces orange juice', 'entities_en': [[0, 1, 'QUANTITY'], [2, 8, 'UNIT'], [9, 12, 'FOOD'], [13, 14, 'QUANTITY'], [15, 21, 'UNIT'], [22, 32, 'FOOD'], [33, 34, 'QUANTITY'], [35, 41, 'UNIT'], [42, 51, 'FOOD'], [52, 54, 'QUANTITY'], [55, 61, 'UNIT'], [62, 74, 'FOOD']], 'text_it': "5 once rum;4 once triplo sec;3 once Zia Maria;20 once succo d'arancia", 'entities_it': [[0, 1, 'QUANTITY'], [2, 6, 'UNIT'], [7, 10, 'FOOD'], [11, 12, 'QUANTITY'], [13, 17, 'UNIT'], [18, 28, 'FOOD'], [29, 30, 'QUANTITY'], [31, 35, 'UNIT'], [36, 45, 'FOOD'], [46, 48, 'QUANTITY'], [49, 53, 'UNIT'], [54, 69, 'FOOD']]}, {'text_en': '2 tubes cinnamon roll, refrigerated, with icing;4 tablespoons butter, melted;6 eggs;1/2 cup milk;2 teaspoons cinnamon;2 teaspoons vanilla;1 cup maple syrup', 'entities_en': [[0, 1, 'QUANTITY'], [2, 7, 'UNIT'], [8, 21, 'FOOD'], [23, 35, 'PROCESS'], [37, 41, 'FOOD'], [42, 47, 'FOOD'], [48, 49, 'QUANTITY'], [50, 61, '

In [4]:
# check if the file is fine
for i, recipe in enumerate(recipe_list):
    print(i, '------------------')
    print("recipe['text_en']", recipe['text_en'])
    print("recipe['text_it']", recipe['text_it'])
    for entity in recipe['entities_it']:
        # print using the original separator
        print(recipe['text_it'][entity[0]:entity[1]+1].strip(), end = ' ')
    print()


0 ------------------
recipe['text_en'] 5 ounces rum;4 ounces triple sec;3 ounces Tia Maria;20 ounces orange juice
recipe['text_it'] 5 once rum;4 once triplo sec;3 once Zia Maria;20 once succo d'arancia
5 once rum; 4 once triplo sec; 3 once Zia Maria; 20 once succo d'arancia 
1 ------------------
recipe['text_en'] 2 tubes cinnamon roll, refrigerated, with icing;4 tablespoons butter, melted;6 eggs;1/2 cup milk;2 teaspoons cinnamon;2 teaspoons vanilla;1 cup maple syrup
recipe['text_it'] 2 tubi rotolo alla cannella, refrigerato, con glassa;4 cucchiai butter, fuso;6 uova;1/2 tazza latte;2 cucchiaini cannella;2 cucchiaini vaniglia;1 tazza sciroppo d'acero
2 tubi rotolo alla cannella, refrigerato, con glassa; 4 cucchiai butter, fuso; 6 uova; 1/2 tazza latte; 2 cucchiaini cannella; 2 cucchiaini vaniglia; 1 tazza sciroppo d'acero 
2 ------------------
recipe['text_en'] 4 ripe coconuts;1 cup evaporated milk;1 cup gin;3 tablespoons sugar (optional);1 teaspoon ground cinnamon;1/2 teaspoon freshly 

In [5]:
new_data = {'classes': data['classes'], 'annotations': recipe_list}

with open(json_path.replace('.json', '_en-it_itemwise_deepl.json'), 'w') as json_file:
    json.dump(new_data, json_file)